In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%%capture
!pip install wandb transformers ml_collections
!wandb login 8c920c1bbdcc7ee5353aaa8becd54a4942f0fc06
!wget https://codalab.lisn.upsaclay.fr/my/datasets/download/3e22f138-ca00-4b10-a0fd-2e914892200d
!unzip /content/3e22f138-ca00-4b10-a0fd-2e914892200d

In [ ]:
from transformers import AutoTokenizer, AutoModel
import json
import random
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from copy import deepcopy
import torch
from torch import nn


def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

In [ ]:
import numpy as np
from sklearn.metrics import f1_score

def accuracy(true, pred):
    true = np.array(true)
    pred = np.array(pred)
    acc = np.sum((true == pred).astype(np.float32)) / len(true)
    return acc * 100

def macro_f1(true, pred):
    true = np.array(true)
    pred = np.array(pred)
    f1 = f1_score(true, pred, average="binary")
    return f1

In [ ]:
YEARWISE_TOKENIZER = {
    "2019": AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-2019-90m"),
    "2020": AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-dec2020"),
    "2021": AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-2021-124m"),
    "2022": AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-jun2022")
}

YEARWISE_MODEL = {
    "2019": AutoModel.from_pretrained("cardiffnlp/twitter-roberta-base-2019-90m"),
    "2020": AutoModel.from_pretrained("cardiffnlp/twitter-roberta-base-dec2020"),
    "2021": AutoModel.from_pretrained("cardiffnlp/twitter-roberta-base-2021-124m"),
    "2022": AutoModel.from_pretrained("cardiffnlp/twitter-roberta-base-jun2022")
}

Downloading:   0%|          | 0.00/341 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-2021-124m were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-2021-124m and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for p

Downloading:   0%|          | 0.00/732 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-jun2022 were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-jun2022 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predi

In [ ]:
import re
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F1F2-\U0001F1F4"  # Macau flag
        u"\U0001F1E6-\U0001F1FF"  # flags
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", flags=re.UNICODE)

# text =

### Dataset class

In [ ]:
print(YEARWISE_TOKENIZER["2019"](text="'containment'",
                                add_special_tokens=False,
                                return_attention_mask=True,
                                truncation=True,
                                return_tensors="pt"))
print(YEARWISE_TOKENIZER["2019"](text="containment",
                                add_special_tokens=False,
                                return_attention_mask=True,
                                truncation=True,
                                return_tensors="pt"))

{'input_ids': tensor([[  108, 10800, 26741,   108]]), 'attention_mask': tensor([[1, 1, 1, 1]])}
{'input_ids': tensor([[10800, 26741]]), 'attention_mask': tensor([[1, 1]])}


In [ ]:
%%capture
!pip install cleantext

from cleantext import clean

    
# text = give_emoji_free_text(text)

In [ ]:
import string

def exists(*args):
    for i in args:
        if i is None:
            return False
    return True

class EvoNLPDataset(Dataset):

    @staticmethod
    def load_instances(fn="/content/drive/MyDrive/evonlp/data/data/train.data.jl"):
        instances = []
        with open(fn) as f:
            for jl_str in f:
                instances.append(json.loads(jl_str))
        
        return instances
    
    @staticmethod
    def load_labels(fn="/content/drive/MyDrive/evonlp/data/data/train.data.jl"):
        fn = fn.replace("data.jl", "labels.tsv")
        labels = dict()
        with open(fn) as f:
            for jl_str in f.readlines():
                id, lbl = jl_str.split("\t")
                labels[id] = lbl
        
        return labels

    def __init__(self, filepath, tokenizer, test=False):
        self.filepath = filepath
        self.instances = EvoNLPDataset.load_instances(fn=self.filepath)
        
        self.tokenizer = tokenizer
        self.test = test

        if not self.test:
            self.labels = EvoNLPDataset.load_labels(fn=self.filepath)
        else:
            self.labels=None

    def __len__(self):
        return len(self.instances)
    
    def find_loc(self, tweet, word):
        for idx in range(len(tweet["input_ids"][0]) - len(word["input_ids"][0])):
            if torch.all(tweet["input_ids"][0][idx: idx+len(word["input_ids"][0])] == word["input_ids"][0]):
                return idx


    def __getitem__(self, idx):
        # First check if word is at the start or in the middle of the sentence
        # for both tweets

        instance = self.instances[idx]
        inst_id = instance["id"]
        word = instance["word"]

        word1 = deepcopy(word)
        word2 = deepcopy(word)

        tweet1 =  emoji_pattern.sub(r'', instance["tweet1"]["text"])
        tweet2 =  emoji_pattern.sub(r'', instance["tweet2"]["text"])
        # tweet1 = tweet1.translate(str.maketrans('', '', string.punctuation))
        # tweet2 = tweet2.translate(str.maketrans('', '', string.punctuation))
        tweet1 = re.sub(r'[^\w\s]', ' ', tweet1)
        tweet2 = re.sub(r'[^\w\s]', ' ', tweet2)
        # tweet2 = clean(instance["tweet2"]["text"], no_emoji=True)

        year_1 = instance["tweet1"]['date'].split('-')[0]
        year_2 = instance["tweet2"]['date'].split('-')[0]
        
        non_spaced_word1 = word1
        non_spaced_word2 = word2

        if tweet1.find(word1) > 0:
            word1 =  " " + word1

        if tweet2.find(word2) > 0:
            word2 =  " " + word2



        # Tokenize tweets and words
        tweet1 = self.tokenizer[year_1](text=tweet1,
                                padding="max_length",
                                max_length=256,
                                add_special_tokens=True,
                                return_attention_mask=False,
                                truncation=True,
                                return_tensors="pt")

        tweet2 = self.tokenizer[year_2](text=tweet2,
                                padding="max_length",
                                max_length=256,
                                add_special_tokens=True,
                                return_attention_mask=False,
                                truncation=True,
                                return_tensors="pt")

        word1 = self.tokenizer[year_1](text=word1,
                                add_special_tokens=False,
                                return_attention_mask=True,
                                truncation=True,
                                return_tensors="pt")

        word2 = self.tokenizer[year_2](text=word2,
                                add_special_tokens=False,
                                return_attention_mask=True,
                                truncation=True,
                                return_tensors="pt")

        non_spaced_word1 = self.tokenizer[year_1](text=non_spaced_word1,
                                add_special_tokens=False,
                                return_attention_mask=True,
                                truncation=True,
                                return_tensors="pt")

        non_spaced_word2 = self.tokenizer[year_2](text=non_spaced_word2,
                                add_special_tokens=False,
                                return_attention_mask=True,
                                truncation=True,
                                return_tensors="pt")

        s_added_word1 = self.tokenizer[year_1](text=word + "s",
                                add_special_tokens=False,
                                return_attention_mask=True,
                                truncation=True,
                                return_tensors="pt")

        s_added_word2 = self.tokenizer[year_2](text=word + "s",
                                add_special_tokens=False,
                                return_attention_mask=True,
                                truncation=True,
                                return_tensors="pt")

        spaced_s_added_word1 = self.tokenizer[year_1](text=" " + word + "s",
                                add_special_tokens=False,
                                return_attention_mask=True,
                                truncation=True,
                                return_tensors="pt")

        spaced_s_added_word2 = self.tokenizer[year_2](text=" " + word + "s",
                                add_special_tokens=False,
                                return_attention_mask=True,
                                truncation=True,
                                return_tensors="pt")

        later_spaced_s_added_word1 = self.tokenizer[year_1](text=" " + word + "s",
                                add_special_tokens=False,
                                return_attention_mask=True,
                                truncation=True,
                                return_tensors="pt")

        later_spaced_s_added_word2 = self.tokenizer[year_2](text=" " + word + "s",
                                add_special_tokens=False,
                                return_attention_mask=True,
                                truncation=True,
                                return_tensors="pt")

        # tweet1 # [1,2,3,4,5,6]
        # word1 # [2,3]
        loc1 = self.find_loc(tweet1, word1)
        loc2 = self.find_loc(tweet2, word2)

        if loc1 is None:
            loc1 = self.find_loc(tweet1, non_spaced_word1)
        
        if loc2 is None:
            loc2 =  self.find_loc(tweet2, non_spaced_word2)

        if loc1 is None:
            loc1 = self.find_loc(tweet1, s_added_word1)
        
        if loc2 is None:
            loc2 =  self.find_loc(tweet2, s_added_word2)
        
        if loc1 is None:
            loc1 = self.find_loc(tweet1, spaced_s_added_word1)
        
        if loc2 is None:
            loc2 =  self.find_loc(tweet2, spaced_s_added_word2)

        if loc1 is None:
          print(instance["tweet1"])
          print(instance["word"])

        if loc2 is None:
          print(instance["tweet2"])
          print(instance["word"])

        if self.test:
            return inst_id, tweet1, tweet2, loc1, loc2, len(word1["input_ids"][0]), len(word2["input_ids"][0]), year_1, year_2, inst_id
        else:
            if loc1 is None or loc2 is None:
                print()
            return inst_id, tweet1, tweet2, loc1, loc2, len(word1["input_ids"][0]), len(word2["input_ids"][0]), year_1, year_2, int(self.labels[inst_id].rstrip("\n")), inst_id

In [ ]:
train_ds = EvoNLPDataset("/content/TempoWiC_Starting_Kit/data/train.data.jl", YEARWISE_TOKENIZER, test=False)
# train_ds[0]
val_ds = EvoNLPDataset("/content/TempoWiC_Starting_Kit/data/validation.data.jl", YEARWISE_TOKENIZER, test=False)

train_dl = DataLoader(train_ds, batch_size=64, shuffle=True, drop_last=True)
val_dl = DataLoader(val_ds, batch_size=64, shuffle=False, drop_last=True)

### Model

In [ ]:
class EvoThresholdModel(nn.Module):
    def __init__(self, freeze_backbone=False, threshold=0.9, test=False):
        super().__init__()
        self.models = YEARWISE_MODEL
        for k, v in self.models.items():
            self.models[k] = v.to(device)
        self.threshold = threshold
        self.test= test

        if freeze_backbone:
            self.freeze_layers()

        

    #  [tensor([ 4, 27,  6, 19,  9, 42, 11,  9]), tensor([ 6, 29,  8, 20, 11, 44, 12, 11])],  # word1 (start_tensor, end_tensor)
    #  [tensor([19,  4, 16,  9, 28, 28, 10, 11]), tensor([21,  6, 18, 10, 30, 30, 11, 13])]   # word2 (start_tensor, end_tensor)
    def freeze_layers(self):
        for model in self.models.values():
            for param in model.parameters():
                param.requires_grad = False
        
    def forward(self, batch):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

        # last_hidden_state pooler_output hidden_states 

        if not self.test:
            tweet1_outputs = self.models[batch[7][0]](input_ids=batch[1]["input_ids"].squeeze().to(device), output_hidden_states=True)
            tweet2_outputs = self.models[batch[8][0]](input_ids=batch[2]["input_ids"].squeeze().to(device), output_hidden_states=True)
        else:
            # print(batch[1]["input_ids"].shape)
            tweet1_outputs = self.models[batch[7][0]](input_ids=batch[1]["input_ids"][0].to(device), output_hidden_states=True)
            tweet2_outputs = self.models[batch[8][0]](input_ids=batch[2]["input_ids"][0].to(device), output_hidden_states=True)
            
        cos_dists = []

        tweet1_hidden_states = tweet1_outputs.hidden_states
        tweet2_hidden_states = tweet2_outputs.hidden_states

        # print(tweet1_outputs.last_hidden_state.shape)

        # tweet1_pooler_op = tweet1_outputs.last_hidden_state[:,0,:]
        # tweet2_pooler_op = tweet2_outputs.last_hidden_state[:,0,:]

        
        tweet1_pooler_op = tweet1_outputs.pooler_output
        tweet2_pooler_op = tweet2_outputs.pooler_output


        for idx in range(len(batch[-1])):

            start1 = batch[3][idx]
            end1 = start1 + batch[5][idx]
            
            start2 = batch[4][idx]
            end2 = start2 + batch[6][idx]
            word1 = []
            word2 = []
            for layer_idx in range(4):
                word1.append(tweet1_hidden_states[layer_idx][idx][start1:end1])
                word2.append(tweet2_hidden_states[layer_idx][idx][start2:end2])
            
            
            word1 = torch.concat(word1, dim=-1)
            word2 = torch.concat(word2, dim=-1)
            

            word1 = torch.sum(word1, dim=0)
            word2 = torch.sum(word2, dim=0)

            word1 = torch.concat([word1, tweet1_pooler_op[idx]], dim=-1)
            word2 = torch.concat([word2, tweet2_pooler_op[idx]], dim=-1)
            
            # tok_len, feat_dim
            # print(word1.shape)

            # print(word1.shape)
            
            cos_distance = torch.nn.functional.cosine_similarity(word1, word2, dim=-1, eps=1e-08)
            # print(cos_distance.shape)

            cos_dists.append(cos_distance)
        return torch.stack(cos_dists)

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

# del model
torch.cuda.empty_cache()
model = EvoThresholdModel(freeze_backbone=True, threshold=0.9, test=False)

model.to(device)

EvoThresholdModel()

### Training loop

In [ ]:
classname = model.__class__.__name__

cos_dists = []
labels = []


model.eval()
with torch.no_grad():
    for idx, batch in enumerate(train_dl):
        print(idx, end=" ")
        outputs = model(batch)
        if classname is "EvoModel":
            cos_dists.append(outputs.argmax(dim=-1).cpu())
        else:
            cos_dists.append(outputs.cpu())
        labels.append(batch[-2].cpu())
        
    
    print()

    # for idx, batch in enumerate(val_dl):
    #     print(idx, end=" ")
    #     outputs = model(batch)
    #     if classname is "EvoModel":
    #         cos_dists.append(outputs.argmax(dim=-1).cpu())
    #     else:
    #         cos_dists.append(outputs.cpu())
    #     labels.append(batch[-1].cpu())

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 


In [ ]:
cos_dists = torch.concat(cos_dists, dim=-1)
labels = torch.concat(labels, dim=-1)

In [ ]:
accs = []
f1_scores = []
for threshold in np.arange(0., 1., 0.001):
    # print([1 if x > threshold else 0 for x in cos_dists])
    preds = np.array([1 if x > threshold else 0 for x in cos_dists])
    f1_scores.append((threshold, f1_score(labels, preds)))
    accs.append((threshold, accuracy(labels, preds)))

sorted_f1 = sorted(f1_scores, key=lambda x: x[1], reverse=True)
sorted_acc = sorted(accs, key=lambda x: x[1], reverse=True)

In [ ]:
print(list(cos_dists.cpu().numpy()))

[0.7748039, 0.71454287, 0.82757896, 0.73387617, 0.8090608, 0.7411841, 0.7691071, 0.8194354, 0.69334126, 0.8598614, 0.5860093, 0.7697995, 0.770732, 0.8148761, 0.83137095, 0.8079642, 0.71138936, 0.7414763, 0.8273962, 0.8314824, 0.71120363, 0.886411, 0.7988529, 0.8057662, 0.79993, 0.81648064, 0.7390304, 0.6899322, 0.7499336, 0.72651887, 0.8652048, 0.79949534, 0.79110736, 0.82129747, 0.8009778, 0.7619033, 0.76845264, 0.7723155, 0.80571616, 0.80438006, 0.8553207, 0.82372487, 0.851433, 0.7995031, 0.8125029, 0.7372112, 0.85497105, 0.8232883, 0.8209436, 0.79554176, 0.8156955, 0.8293524, 0.81882745, 0.76150143, 0.74054813, 0.8389786, 0.7127938, 0.778655, 0.6984378, 0.7682262, 0.86743, 0.8344097, 0.7160429, 0.7691908, 0.81846035, 0.7852786, 0.800719, 0.8618467, 0.73742914, 0.8195704, 0.80825335, 0.7673143, 0.72200793, 0.8020861, 0.88326263, 0.860657, 0.7247976, 0.8258165, 0.81871593, 0.77040064, 0.7792201, 0.8442962, 0.8276882, 0.68583345, 0.7194716, 0.7691033, 0.7481835, 0.8508805, 0.7827295, 0

In [ ]:
print(sorted_f1)

[(0.587, 0.6264705882352941), (0.588, 0.6264705882352941), (0.589, 0.6264705882352941), (0.59, 0.6264705882352941), (0.591, 0.6264705882352941), (0.592, 0.6264705882352941), (0.593, 0.6264705882352941), (0.594, 0.6264705882352941), (0.595, 0.6264705882352941), (0.596, 0.6264705882352941), (0.597, 0.6264705882352941), (0.598, 0.6264705882352941), (0.599, 0.6264705882352941), (0.6, 0.6264705882352941), (0.601, 0.6264705882352941), (0.602, 0.6264705882352941), (0.603, 0.6264705882352941), (0.604, 0.6264705882352941), (0.605, 0.6264705882352941), (0.606, 0.6264705882352941), (0.607, 0.6264705882352941), (0.608, 0.6264705882352941), (0.609, 0.6264705882352941), (0.61, 0.6264705882352941), (0.611, 0.6264705882352941), (0.612, 0.6264705882352941), (0.613, 0.6264705882352941), (0.614, 0.6264705882352941), (0.615, 0.6264705882352941), (0.616, 0.6264705882352941), (0.617, 0.6264705882352941), (0.618, 0.6264705882352941), (0.619, 0.6264705882352941), (0.62, 0.6264705882352941), (0.621, 0.62647058

In [ ]:
print(sorted_acc)

[(0.895, 54.82954545454546), (0.896, 54.82954545454546), (0.903, 54.82954545454546), (0.904, 54.82954545454546), (0.897, 54.75852272727273), (0.899, 54.75852272727273), (0.902, 54.75852272727273), (0.905, 54.75852272727273), (0.894, 54.6875), (0.898, 54.6875), (0.9, 54.6875), (0.901, 54.6875), (0.906, 54.6875), (0.907, 54.6875), (0.908, 54.6875), (0.909, 54.6875), (0.91, 54.6875), (0.911, 54.6875), (0.886, 54.61647727272727), (0.887, 54.61647727272727), (0.89, 54.61647727272727), (0.891, 54.61647727272727), (0.892, 54.61647727272727), (0.893, 54.61647727272727), (0.912, 54.61647727272727), (0.913, 54.61647727272727), (0.914, 54.61647727272727), (0.915, 54.61647727272727), (0.916, 54.61647727272727), (0.917, 54.61647727272727), (0.918, 54.61647727272727), (0.919, 54.61647727272727), (0.92, 54.61647727272727), (0.921, 54.61647727272727), (0.922, 54.61647727272727), (0.923, 54.61647727272727), (0.924, 54.61647727272727), (0.925, 54.61647727272727), (0.926, 54.61647727272727), (0.927, 54.6

In [ ]:
# THRESHOLD = 0.917
THRESHOLD = sorted_f1[0][0]


### Evaluation

In [ ]:
test_ds = EvoNLPDataset("/content/TempoWiC_Starting_Kit/data/test-codalab-10k.data.jl", YEARWISE_TOKENIZER, test=True)
test_dl = DataLoader(test_ds, batch_size=1, shuffle=False, drop_last=False)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

#del model
torch.cuda.empty_cache()
model = EvoThresholdModel(freeze_backbone=True, threshold=0.9, test=True)

model.to(device)

EvoThresholdModel()

In [ ]:
classname = model.__class__.__name__

cos_dists = []
labels = []
inst_ids = []

model.eval()
with torch.no_grad():
    for idx, batch in enumerate(test_dl):
        print(idx, end=" ")
        outputs = model(batch)
        if classname is "EvoModel":
            cos_dists.append(outputs.argmax(dim=-1).cpu())
            inst_ids.append(batch[-1][0])
        else:
            cos_dists.append(outputs.cpu())
            inst_ids.append(batch[-1][0])

        # labels.append(batch[-1].cpu())
torch.save(cos_dists, "test_set_so_dists.pt")

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [ ]:
import pandas as pd

preds = np.array([0 if x > THRESHOLD else 1 for x in cos_dists])


preds = pd.DataFrame(list(zip(inst_ids, preds)), columns =['Id', 'Prediction'])
preds.to_csv('/content/drive/MyDrive/evonlp/data/timelms_cosine_v4.tsv', sep="\t",header=False, index = False)

In [ ]:
# def load_instances(fn="/content/TempoWiC_Starting_Kit/data/test-codalab-10k.data.jl"):
#     instances = []
#     with open(fn) as f:
#         for jl_str in f:
#             instances.append(json.loads(jl_str))
    
#     return instances
# id_list =[]
# instance = load_instances()

# for i in instance:
#   id_list.append(i["id"])

# cos_dists = torch.concat(cos_dists, dim=0)
# # labels = torch.concat(labels, dim=0)

# # print(cos_dists.shape)
# # print(labels.shape)

# cos_dists = cos_dists.numpy()
# # labels = labels.numpy()

# accs = []
# f1_scores = []

# sorted_f1 = sorted(f1_scores, key=lambda x: x[1], reverse=True)
# sorted_acc = sorted(accs, key=lambda x: x[1], reverse=True)